In [2]:
%pip install emoji spacy
#!python -m spacy download es_core_news_sm

#%pip install python-louvain


     ------------------------------------- 608.4/608.4 kB 19.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [50]:
%pip uninstall pyvis --yes
%pip install pyvis==0.3.1

Found existing installation: pyvis 0.3.2
Uninstalling pyvis-0.3.2:
  Successfully uninstalled pyvis-0.3.2
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/748.9 kB ? eta -:--:--
     ------------------------------------- 748.9/748.9 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyvis: filename=pyvis-0.3.1-py3-none-any.whl size=755933 sha256=ae46c40089b8791d4d34097cbda54af78ec51b68062c10cd997f50d693cc045d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\37\f9\93\44dd6cbfb2ead35307b114d27af8c6a14d5762a462af1e04f5
Successfully built pyvis
Note: you may need to restart the kernel to

In [22]:
%matplotlib inline

In [3]:
import pandas as pd
import re, emoji, spacy, numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx
from pyvis.network import Network
import community
import matplotlib.colors as mcolors
import heapq
import webbrowser



ModuleNotFoundError: No module named 'networkx'

In [30]:
# Cargar spaCy sin NER ni parser (más rápido)
nlp = spacy.load("es_core_news_sm", disable=["ner", "parser"])

In [25]:
df = pd.read_csv("../data/processed/2_comments_youtube_refined.csv")

# Nube de Palabras

In [118]:
# 1) Lista manual de stopwords en español (puedes ampliarla si quieres)
spanish_stopwords = [
    "de","la","que","el","en","y","a","los","del","se","las","por","un","para",
    "con","no","una","su","al","lo","como","más","pero","sus","le","ya","o","este",
    "sí","porque","esta","entre","cuando","muy","sin","sobre","también","me","hasta",
    "hay","donde","quien","desde","todo","nos","durante","todos","uno","les","ni",
    "contra","otros","ese","eso","ante","ellos","e","esto","mí","antes","algunos",
    "qué","unos","yo","otro","otras","otra","él","tanto","esa","estos","mucho",
    "quienes","nada","muchos","cual","poco","ella","estar","estas","algunas","algo",
    "nosotros","mi","mis","tú","te","ti","tu","tus","ellas","nosotras","vosotros",
    "vosotras","os","mío","mía","míos","mías","tuyo","tuya","tuyos","tuyas","suyo",
    "suya","suyos","suyas","nuestro","nuestra","nuestros","nuestras","vuestro",
    "vuestra","vuestros","vuestras","esos","esas", "es", "ser", "fue", "son", "sido", "tiene", "tenido", "tienen", "tenía",
    "tenían", "tendría", "tendrían", "había", "habían", "habrá", "habrán", "habría", "son", "serán", "sería", "serían",
    "está", "están", "estuve", "estuvieron", "estaría", "estarían", "esté", "estén", "estuviera", "estuvieran", 
    "mas", "q", "solo", "tiene", "tienen", "si", "no", "estan", "va", "ha", "vivo", "viva", "ahora", "asi", "aqui", "ahi", 
    "siempre", "nunca", "ya", "sabe", "misma", "tambien", "era", "ve", "dio", "fueron", "toda", "misma", "hacer", "entonces", 
    "iran", "cosa", "dice", "querer", "poder",  "haber",  "ver", "tener", "dar", "deber", "creer", "saber", "seguir", "mismo", "bien", 
    "ir", "ganar", "perder", "dejar",  "decir" 
]

In [119]:
nlp = spacy.load("es_core_news_sm", disable=["ner", "parser"])

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)                      # eliminar URLs
    text = emoji.replace_emoji(text, replace="")             # eliminar emojis
    text = re.sub(r"\d+", "", text)                          # eliminar números
    text = re.sub(r"[^a-záéíóúüñ\s]", "", text)              # eliminar signos
    text = re.sub(r"\s+", " ", text).strip()                 # espacios múltiples
    return text

df['clean_comment'] = df['comment'].astype(str).apply(preprocess_text)

def lemmatize_texts(texts, stopwords):
    docs = nlp.pipe(texts, batch_size=1000)
    result = []
    for doc in docs:
        tokens = [t.lemma_ for t in doc if t.lemma_ not in stopwords and not t.is_punct]
        result.append(" ".join(tokens))
    return result

df['clean_comment'] = lemmatize_texts(df['clean_comment'], spanish_stopwords)
# Corrección manual post-lematización
df['clean_comment'] = df['clean_comment'].str.replace(r'\botar\b', 'otan', regex=True)

# Filtrado adicional de stopwords persistentes
remove_words = ['él', 'ella', 'ellos', 'ellas', 'nosotros', 'vosotros', 'tú', 'yo']
pattern = r'\\b(?:' + '|'.join(remove_words) + r')\\b'
df['clean_comment'] = df['clean_comment'].str.replace(pattern, '', regex=True)
df['clean_comment'] = df['clean_comment'].str.replace(r'\s+', ' ', regex=True).str.strip()




In [112]:
vectorizer = CountVectorizer(min_df=200, max_df=0.8, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_comment'])
words = vectorizer.get_feature_names_out()
cooc_matrix = (X.T @ X).toarray()
np.fill_diagonal(cooc_matrix, 0)
word_freq = X.sum(axis=0).A1

top_n = 30
top_indices = heapq.nlargest(top_n, range(len(word_freq)), key=lambda i: word_freq[i])

G = nx.Graph()
for i in top_indices:
    G.add_node(words[i], size=int(word_freq[i]))

threshold = 200
for i in top_indices:
    for j in top_indices:
        if i < j and cooc_matrix[i, j] > threshold:
            G.add_edge(words[i], words[j], weight=int(cooc_matrix[i, j]))

G.remove_nodes_from(list(nx.isolates(G)))


In [110]:
partition = community.best_partition(G)
for node, comm_id in partition.items():
    G.nodes[node]['group'] = comm_id

palette = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())
net = Network(height="850px", width="100%", notebook=False)

# 💡 USAR barnes_hut: mejor dispersión
net.barnes_hut(
    gravity=-30000,           # fuerza repulsiva fuerte
    central_gravity=0.2,
    spring_length=300,        # separa nodos
    spring_strength=0.005,
    damping=0.9,
    overlap=1                 # evita solapamiento
)

# Colores por comunidad
palette = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())

# Nodos
for node, data in G.nodes(data=True):
    size = int(data.get('size', 1))
    group = data.get('group', 0)
    color = palette[group % len(palette)]
    net.add_node(
        str(node),
        label=str(node),
        title=f"{node} (Frecuencia: {size})",
        size=size,
        color=color,
        font={"size": 22}  # tamaño visible
    )

# Aristas
for u, v, d in G.edges(data=True):
    net.add_edge(
        str(u), str(v),
        value=int(d['weight']),
        title=f"Coocurrencias: {d['weight']}"
    )

net.toggle_physics(True)
net.show_buttons(filter_=['physics'])
net.write_html("grafo_comentarios_final.html")


.......................

In [ ]:
webbrowser.open("grafo_comentarios_final.html")

True

In [113]:
terms = vectorizer.get_feature_names_out()
frequencies = X.sum(axis=0).A1

df_freq = pd.DataFrame({
    "termino": terms,
    "frecuencia": frequencies
})
df_freq["tipo"] = df_freq["termino"].apply(lambda x: "Bigrama" if " " in x else "Unigrama")
df_freq = df_freq.sort_values(by="frecuencia", ascending=False)
df_freq.head(30)



,termino,frecuencia,tipo
752,rusia,29700,Unigrama
872,ucrania,21855,Unigrama
377,guerra,15888,Unigrama
762,ruso,12887,Unigrama
929,él,10477,Unigrama
706,putin,8683,Unigrama
876,ucraniano,7195,Unigrama
639,país,6369,Unigrama
543,miguel,6223,Unigrama
618,otar,5890,Unigrama


In [106]:
df_bigrams = df_freq[df_freq["tipo"] == "Bigrama"]
df_bigrams.head(20)


,termino,frecuencia,tipo
639,estados unidos,1979,Bigrama
1561,rusia tener,1348,Bigrama
1551,rusia ir,1044,Bigrama
740,ganar guerra,1037,Bigrama
789,guerra mundial,979,Bigrama
924,ir ganar,970,Bigrama
1086,memoria pez,962,Bigrama
1810,ucrania ir,802,Bigrama
1818,ucrania tener,780,Bigrama
638,estados unido,728,Bigrama


OTAR? 

In [117]:
df_otar = df[df['clean_comment'].str.contains(r'\botar\b')]
df_otar[['comment', 'clean_comment']].sample(10)

,comment,clean_comment
97377,Eso quiere informar a toda OTAN que ya termino...,informar otar terminar entrenamiento listo gue...
72258,❤OTAN NATO OCCIDENTE TIENEN QUE ABANDONAR UCR...,otar nato occidente abandonar ucrania inmediat...
1243,Rusia no ha ganado nada porque los paises de l...,rusia país otar cada dia arma mento calidad ru...
40105,LOS RUSOS ROGANDO a los países que ayudan a Uc...,ruso rogar país ayudar ucrania rogar pues juga...
44274,Si los ucranianos que luchan por el régi men a...,ucraniano luchar régi men atacar suelo ruso ar...
41309,El ejercito ucraniano defendiendo los interese...,ejercito ucraniano defender interés oro estado...
12560,Los que empesaron usando bombas de racimo y ah...,empesar usar bomba racimo fosforo otar destrui...
49036,"Si Rusia fuese como Israel, hace mucho tiempo ...",rusia israel tiempo tomar kiev diferencia tier...
22623,😂😂😂 Rusia no le da ni por los tobillos al pode...,rusia tobillo poderio otar ucraniano otan derr...
25926,La unión europea debería crear un ente como la...,unión europea crear ente otar estados unidos a...


In [115]:
print(f"Total comentarios con 'otar': {len(df_otar)}")

# Si tenés columna de usuario
if "user_id" in df_otar.columns:
    print(f"Usuarios únicos: {df_otar['user_id'].nunique()}")
    print("Top usuarios que lo usan:")
    print(df_otar['user_id'].value_counts().head(10))

# Ver muestra de comentarios
df_otar['comment'].sample(10).tolist()


Total comentarios con 'otar': 5354
Usuarios únicos: 4321
Top usuarios que lo usan:
user_id
UCDPXyG6e1vn13TEr7tUPzgQ    33
UCq5zsco1r4DN2rudCE5wIcg    18
UC7AIJAb1ql4WsirczQy3f9w    16
UC3qJAsZBq378niGl0dUkdEg    15
UCRvhxzKkHPR25yloobXiRrA    14
UCd4l7HA1A7TYeubDfH6PPxg    14
UCWjxPMfIGGdfN29D5xXOHsQ    10
UCdT9jgMZVZU-c4o0MI-fK9g    10
UCiW_XDALSnOFxZXr3c6FYeA    10
UCyiSZNQX3jGYmiU-RbAY0KA     9
Name: count, dtype: int64


['Desde cuando las fuerzas ucranianas lanzan misiles norteamericanos contra Rusia? Lo dices y te quedas tan fresco! Eres un desinformador intoxicador de la opinión pública que ocultas la responsabilidad directa de la OTAN en lo que ocurre y lo que va a ocurrir!',
 'No digas Ucrania... Mejor di Otan',
 'Yo creo son , solo palabras, si la OTAN y EU quisiera actuar ya lo vieran hecho. Solo dan falsas esperanzas a ucrania.',
 'Se repetirá la historia y la otan será aniquilados  por completo de eso estoy bien seguro',
 'Cual seria la situación actual si Estados Unidos y la OTAN no hubiera influenciado en Ucrania?',
 'Muchos han sido forzados a Unirse al Ejército Ruso.\nFuente: La Otan.',
 'Se habla de la intervención de Rusia en Ucrania como sia Rusia no tuviera costo humano de su parte para mantener la existencia de Rusia y Rusia dentro de su terreno mientras q la OTAN expandiendose e invadiendo, corroyendo los estados para su beneficio.',
 'Miguel toda esa cosa pasa por putin por ejenplo 

In [116]:
vectorizer_otar = CountVectorizer(min_df=10, max_df=0.9, ngram_range=(1, 2))
X_otar = vectorizer_otar.fit_transform(df_otar['clean_comment'])

terms = vectorizer_otar.get_feature_names_out()
freqs = X_otar.sum(axis=0).A1

df_terms_otar = pd.DataFrame({
    "termino": terms,
    "frecuencia": freqs
}).sort_values(by="frecuencia", ascending=False)

df_terms_otar.head(20)


,termino,frecuencia
1626,rusia,3612
1873,ucrania,2548
790,guerra,1698
1671,ruso,1179
2014,él,1096
1372,país,951
569,eeuu,812
1533,putin,745
1254,otan,716
1896,ucraniano,684
